### Set notebook parameters

In [ ]:
dataset_name = 'sl2'
dataset_id = '201030034508'

dataset_path = '/home/ec2-user/SageMaker/lbx-nerf/data/' + dataset_name

### Download source images from S3

In [ ]:
import os

s3uri = 's3://lbxlabs.scandata/' + dataset_id + '/'

os.makedirs(dataset_path, exist_ok=True)

!aws s3 cp $s3uri $dataset_path --recursive

Make sure the dataset folder has the following subfolders:
/home/ec2-user/SageMaker/lbx-nerf/data/{dataset_name} 
```
├── images
└── calibration_data  
    ├── cam_dist_list.npy
    ├── cam_extrinsics.npy 
    ├── cam_locations.npy 
    └── cam_mtx_list.npy 
```

### Generate poses

In [ ]:
import numpy as np 

intrinsics = np.load(dataset_path + '/calibration_data/cam_mtx_list.npy')
extrinsics = np.load(dataset_path + '/calibration_data/cam_extrinsics.npy')
locations = np.load(dataset_path + '/calibration_data/cam_locations.npy')
print(f'Intrinsics have shape: {intrinsics.shape}')
print(f'Extrinsics have shape: {extrinsics.shape}')
print(f'Locations have shape: {locations.shape}')

# Eventually, we can do everything in a vectorized way, but for now, to make things simple,
# lets break apart each matrix
intrinsics = [intr for intr in intrinsics]
extrinsics = [ext for ext in extrinsics]
locations = [loc for loc in locations]

W = 6000
H = 4000
num_camera_columns = 120

# Cosine and sine of 180 degrees
c = -1 
s = 0
rot_x = np.array([
    [1, 0, 0, 0],
    [0, c, -s, 0],
    [0, s, c, 0],
    [0, 0, 0, 1]
])

bounds_w = 12 # Value is in cm, relative to camera-to-origin distance calculated below

poses_bounds = []

for index, (ext, loc) in enumerate(zip(extrinsics, locations)):
    # Figure out which "row" this camera is in: we have to do this because the matrix of
    # camera intrinsics has shape (27, 3, 3), while the other two arrays have shape (3240, ...) - 
    # this is because we only create one intrinsic matrix per "row" of the src cam locations
    row = index // num_camera_columns
    intr = intrinsics[row]
    
    # Apply an additional rotation of 180-degrees about the x-axis (OpenCV -> OpenGL convention)
    #ext = np.matmul(ext, rot_x)
    ext = np.matmul(rot_x, ext)

    # Convert world-to-camera to camera-to-world
    ext = np.linalg.inv(ext)
    print_debug(f'Before \n{ext}')
    
    # Get rid of the last row, which is always(0, 0, 0, 1)
    ext = ext[:3, :]

    # Flip the x and y columns
    ext = ext[:, [1, 0, 2, 3]]

    # Multiply the y column by -1
    ext[:, 0] *= -1
    print_debug(f'After \n{ext}\n')
    
    
    # Append a 5th column to the matrix, which is [h, w, focal]
    focal = (intr[0,0] + intr[1,1]) * 0.5
    last_column = np.array([H, W, focal])
    params = np.c_[ext, last_column]
    print_debug(params)
    

    # Calculate the distance from the camera to the origin using the calibration data
    distance_to_origin = np.linalg.norm(loc)
    print_debug(f'Distance to origin: {distance_to_origin}')

    # The "near" and "far" planes are just set to the distance calculated above +/- a fixed constant
    near = distance_to_origin - bounds_w # np.min(distance_to_origin - bounds_w, loc[1]) # loc[1] is the height of the camera off the stage
    far = 2.5 * distance_to_origin

    # Flatten the 3x5 matrix into a 15-element row vector, append near and far bounds
    params = params.reshape((15,))
    params = np.append(params, [near, far])
    print_debug(f'After reshaping: {params}')
    
    poses_bounds.append(params)

poses_bounds = np.array(poses_bounds)
print(f'All params shape: {poses_bounds.shape}')

# Save .npy file
np.save(dataset_path + '/poses_bounds.npy', poses_bounds)
